In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from common.filter_banks import make_gauss_kernels, make_dog_kernels
from common.filter_banks import wave_numbers, make_sine_kernels, make_gabor_kernels, conv2d_sq
from common.filter_banks import show_filter_bank, show_filter_response
from common.image_ops import resize_img, whiten_img
from imageio import imread
from skimage.color import rgb2gray
from skimage.transform import resize
from skimage import img_as_uint
import cv2
from pathlib import Path
import reprlib, os, logging, sys
log_format = '%(asctime)s | %(levelname)s : %(message)s'
logging.basicConfig(format=log_format, level=logging.INFO, stream=sys.stdout)
logging.info(f"Start with tf version {tf.version.VERSION}")

2020-10-02 21:39:49,093 | INFO : Start with tf version 2.3.1


In [ ]:
%matplotlib notebook
gauss_kernels_tf = make_gauss_kernels()
show_filter_bank(gauss_kernels_tf)

In [ ]:
%matplotlib notebook
dog_kernels_tf = make_dog_kernels()
show_filter_bank(dog_kernels_tf)

In [ ]:
%matplotlib notebook
ks = wave_numbers(6)
freqs=[0.5,1.0,2.0]
xs, ys = grid_for_sz(3)
sine_kernels = []
for freq in freqs:
    for k in ks:
        logging.info(f"k={k} freq={freq}")
        sine_kernels \
            .append(np.exp(freq * (xs*k[0] + ys*k[1]) * 1.0j))
sine_kernels = \
    tf.constant(sine_kernels, tf.complex64)
sine_kernels

In [ ]:
%matplotlib notebook
ks = wave_numbers(6)
gabor_kernels = \
    make_gabor_kernels(sz=13, ks=ks, freqs=[0.5,1.0,2.0])
logging.info(f"{gabor_kernels.shape}")
show_filter_bank(gabor_kernels, rows=3)

## One size to rule them all!

In [2]:
sz=128

In [ ]:
racoon_path = (Path(os.environ['DATA_ALL']) / 'Misc' / 'racoon').with_suffix('.png')
racoon = imread(racoon_path)
racoon = rgb2gray(racoon)
racoon = resize_img(racoon, sz=sz)
racoon = whiten_img(racoon)
racoon = np.reshape(racoon, (1,sz,sz,1))

In [ ]:
%matplotlib notebook
racoon_in_tf = tf.constant(racoon, dtype=tf.float32)
racoon_out_sq = conv2d_sq(racoon_in_tf, gabor_kernels)

# add up the even and odd components
racoon_out_sub = \
    tf.nn.max_pool(racoon_out_sq, (4,4), 
                   strides=2, padding='SAME')

logging.info(racoon_out_sub.shape)
show_filter_response(racoon_out_sub, rows=3)

In [ ]:
%matplotlib notebook
def mse_tf(current_tf:tf.Tensor, kernel_tf:tf.Tensor, match_tf_sub:tf.Tensor):
    current_out_tf_sq = conv2d_sq(current_tf, kernel_tf)    
    current_out_tf_sub = tf.nn.max_pool(current_out_tf_sq, (4,4), strides=2, padding='SAME')    
    loss_per_level = \
        tf.reduce_mean((current_out_tf_sub - match_tf_sub) ** 2, axis=[0,1,2])
    
    # print(loss_per_level.shape)
    loss = tf.tensordot(loss_per_level, 
                        [8.] * 6 + [3.] * 6 + [1.] * 6, 1)
    return loss

current_value = tf.Variable(0.5 * np.ones((1,sz,sz,1)), dtype=tf.float32)
with tf.GradientTape() as tape:
    current_out = mse_tf(current_value, gabor_kernels, racoon_out_sub)
grad = tape.gradient(current_out, current_value)
logging.info(f"Gradient max = {np.max(grad)}")

fig, axs = plt.subplots(1,2,figsize=(5,3))
axs[0].imshow(tf.reshape(current_value, (sz,sz)))
axs[1].imshow(tf.reshape(grad, (sz,sz)))

In [ ]:
%matplotlib notebook
learning_rate = 1e+3
optimizer = tf.optimizers.Adam()
for n in range(1000):
    optimizer.apply_gradients(zip([grad], [current_value]))
    with tf.GradientTape() as tape:
        current_out = mse_tf(current_value, gabor_kernels, racoon_out_sub)
        # tf.print(f"loss = {current_out}   ")
    grad = tape.gradient(current_out, current_value)
    
fig, axs = plt.subplots(1,3,figsize=(5,3))
axs[0].imshow(tf.reshape(racoon[...,0], (sz,sz)), cmap='gray', vmin=0., vmax=0.8)
axs[1].imshow(tf.reshape(current_value, (sz,sz)), cmap='gray', vmin=0., vmax=0.8)
axs[2].imshow(tf.reshape(grad, (sz,sz)))
plt.show()

In [ ]:
%matplotlib notebook
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4,4))
slice_path = Path(os.environ['DATA_ALL']) / 'NIH_DeepLesion' \
                    / 'Images_png_01' / 'Images_png' \
                    / '000001_01_01' / '108.png'
transverse_slice = imread(slice_path)
transverse_slice = resize(transverse_slice, (sz,sz))
transverse_slice = img_as_uint(transverse_slice)
logging.info(np.histogram(transverse_slice, 10))

transverse_slice = clahe.apply(transverse_slice)
ts_min, ts_max = np.min(transverse_slice), np.max(transverse_slice)
ts_width = ts_max - ts_min
transverse_slice = (transverse_slice - ts_min) / ts_width
transverse_slice = np.reshape(transverse_slice, (1,sz,sz,1))
logging.info(np.histogram(transverse_slice, 10))

transverse_slice_tf = tf.constant(transverse_slice, dtype=tf.float32)
transverse_slice_out_tf_sq = conv2d_sq(transverse_slice_tf, gabor_kernels)
transverse_slice_out_sub = \
    tf.nn.max_pool(transverse_slice_out_tf_sq, (2,2), 
                   strides=2, padding='SAME')

show_filter_response(transverse_slice_out_sub, rows=3)

In [ ]:
%matplotlib notebook

clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4,4))
cxr_image_path = Path(os.environ['DATA_ALL']) / 'NIH_Cxr8' \
                        / 'images' / '00010827_000.png'

cxr_image = imread(cxr_image_path)
cxr_image = resize(cxr_image, (sz,sz))
cxr_image = img_as_uint(cxr_image)
logging.info(np.histogram(cxr_image, 10))

cxr_image = clahe.apply(cxr_image)
cx_min, cx_max = np.min(cxr_image), np.max(cxr_image)
cx_width = cx_max - cx_min
cxr_image = (cxr_image - cx_min) / cx_width
cxr_image = np.reshape(cxr_image, (1,sz,sz,1))
logging.info(np.histogram(cxr_image, 10))

cxr_image_tf = tf.constant(cxr_image, dtype=tf.float32)
cxr_image_out_tf_sq = conv2d_sq(cxr_image_tf, gabor_kernels)
cxr_image_out_sub = \
    tf.nn.max_pool(cxr_image_out_tf_sq, (4,4), 
                   strides=2, padding='SAME')

show_filter_response(cxr_image_out_sub, rows=3)

In [ ]:
current_value = tf.Variable(0.5 * np.ones((1,sz,sz,1)), dtype=tf.float32)
with tf.GradientTape() as tape:
    current_out = mse_tf(current_value, gabor_kernels, cxr_image_out_sub)
grad = tape.gradient(current_out, current_value)
print(grad.shape, current_out)
fig, axs = plt.subplots(1,2,figsize=(5,3))
axs[0].imshow(tf.reshape(current_value, (sz,sz)))
axs[1].imshow(tf.reshape(grad, (sz,sz)))

In [ ]:
%matplotlib notebook
from IPython.display import clear_output
learning_rate = 1e+3

optimizer = tf.optimizers.Adam()
for n in range(1000):
    optimizer.apply_gradients(zip([grad], [current_value]))
    # current_value.assign_sub(learning_rate * grad)
    with tf.GradientTape() as tape:
        current_out = mse_tf(current_value, gabor_kernels, cxr_image_out_sub)
        # tf.print(f"loss = {current_out}   ")
    grad = tape.gradient(current_out, current_value)
    
fig, axs = plt.subplots(1,3,figsize=(5,3))
axs[0].imshow(tf.reshape(cxr_image[...,0], (sz,sz)), cmap='gray', vmin=0., vmax=0.8)
axs[1].imshow(tf.reshape(current_value, (sz,sz)), cmap='gray', vmin=0., vmax=0.8)
axs[2].imshow(tf.reshape(grad, (sz,sz)))

In [28]:
from tensorflow.keras.layers import Dense, Input, SpatialDropout2D
from tensorflow.keras.layers import Conv2D, Flatten, Lambda
from tensorflow.keras.layers import LocallyConnected2D, ZeroPadding2D
from tensorflow.keras.layers import MaxPooling2D, UpSampling2D
from tensorflow.keras.layers import Reshape, Conv2DTranspose
from tensorflow.keras.layers import ActivityRegularization
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import mse, binary_crossentropy
from tensorflow.keras import backend as K

def sampling(args):
    """
    Reparameterization trick by sampling fr an isotropic unit Gaussian.
    instead of sampling from Q(z|X), sample eps = N(0,I) 
        then z = z_mean + sqrt(var)*eps    
    # Arguments
        args (tensor tuple): mean and log of variance of Q(z|X)
    # Returns
        z (tensor): sampled latent vector
    """    
    z_mean, z_log_var = args
    
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

def sampling_mean_log_var(z_mean_log_var):
    """ """
    batch, dim = K.shape(z_mean_log_var)[0], K.int_shape(z_mean_log_var)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean_log_var[...,0] + K.exp(0.5 * z_mean_log_var[...,1]) * epsilon


In [29]:
def vae_loss(z_mean, z_log_var, y_true, y_pred):
    """
    Compute VAE loss, using either mse or crossentropy.
    # Arguments
        z_mean: mean of Q(z|X)
        z_log_var: log variance of Q(z|X)
        y_true, y_pred: truth and predicated values
    # Returns
        loss value
    """
    match_loss = mse(K.flatten(y_true), K.flatten(y_pred))
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    return match_loss + (1e-4 * kl_loss)

In [43]:
act_func = 'softplus' # or 'relu'
locally_connected_channels = 2
latent_dim = 8

# this is the V1 filter bank output
v1_filtered_output = Input(shape=(sz//2,sz//2,4), name='v1_{}'.format(sz//2))

encoder = \
    Sequential(name='v1_to_pulvinar_encoder', layers=
    [
        v1_filtered_output,
        Conv2D(64, (1,1), name='v2_conv2d', activation=act_func, padding='same'),
        MaxPooling2D((2,2), name='v2_maxpool', padding='same'),
        Conv2D(64, (3,3), name='v4_conv2d', activation=act_func, padding='same'),
        MaxPooling2D((2,2), name='v4_maxpool', padding='same'),
        Conv2D(64, (1,1), name='pit_conv2d', activation=act_func, padding='same'),
        MaxPooling2D((2,2), name='pit_maxpool', padding='same'),
        Conv2D(64, (3,3), name='cit_conv2d', activation=act_func, padding='same'),
        LocallyConnected2D(locally_connected_channels, (3,3), 
                           name='ait_local', activation=act_func,
                           kernel_regularizer=L1L2(l1=0.01, l2=0.01)),
        Flatten(name='pulvinar_flatten'),
        Dense((latent_dim*2), activation=act_func, name='pulvinar_dense'),
        Reshape((latent_dim,2), name='pulinvar_reshape'),
        Lambda(sampling_mean_log_var, output_shape=(latent_dim,1), name='z')
    ])
encoder_output = encoder(v1_filtered_output)
print(encoder_output)
encoder.summary()

local_shape = encoder.get_layer('ait_local').output_shape
z_shape = encoder.get_layer('z').output_shape
decoder = \
    Sequential(name='pulvinar_to_v1_decoder', layers=
    [
        Input(shape=z_shape, name='z_sampling'),
        Dense(np.prod(local_shape[1:]), name='pulvinar_dense_back', activation=act_func),
        Reshape(local_shape[1:], name='pulvinar_antiflatten'),
        ZeroPadding2D(padding=(1,1), name='ait_padding_back'),
        LocallyConnected2D(locally_connected_channels, (3,3), 
                           name='ait_local_back', activation=act_func,
                           kernel_regularizer=L1L2(l1=0.01, l2=0.01)),
        ZeroPadding2D(padding=(1,1), name='cit_padding_back'),
        Conv2DTranspose(64, (3,3), name='cit_conv2d_trans', activation=act_func, padding='same'),       
        UpSampling2D((2,2), name='cit_upsample_back'),
        Conv2DTranspose(64, (1,1), name='pit_conv2d_trans', activation=act_func, padding='same'),
        UpSampling2D((2,2), name='pit_upsample_back'),
        Conv2DTranspose(64, (3,3), name='v4_conv2d_trans', activation=act_func, padding='same'),
        UpSampling2D((2,2), name='v4_upsample_back'),
        Conv2DTranspose(4, (1,1), name='v2_conv2d_trans', activation=act_func, padding='same')
    ])
decoder_output = decoder(encoder_output)
decoder.summary()

Tensor("v1_to_pulvinar_encoder/z/add_16:0", shape=(None, 8), dtype=float32)
Model: "v1_to_pulvinar_encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
v2_conv2d (Conv2D)           (None, 64, 64, 64)        320       
_________________________________________________________________
v2_maxpool (MaxPooling2D)    (None, 32, 32, 64)        0         
_________________________________________________________________
v4_conv2d (Conv2D)           (None, 32, 32, 64)        36928     
_________________________________________________________________
v4_maxpool (MaxPooling2D)    (None, 16, 16, 64)        0         
_________________________________________________________________
pit_conv2d (Conv2D)          (None, 16, 16, 64)        4160      
_________________________________________________________________
pit_maxpool (MaxPooling2D)   (None, 8, 8, 64)          0         
__________________________________

In [45]:
import pprint
autoencoder = Model(v1_filtered_output, decoder_output, name='v1_to_pulvinar_vae')
autoencoder.summary()
pprint.pprint(autoencoder.trainable_weights)

Model: "v1_to_pulvinar_vae"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
v1_64 (InputLayer)           [(None, 64, 64, 4)]       0         
_________________________________________________________________
v1_to_pulvinar_encoder (Sequ (None, 8)                 121048    
_________________________________________________________________
pulvinar_to_v1_decoder (Sequ (None, 64, 64, 4)         44580     
Total params: 165,628
Trainable params: 165,628
Non-trainable params: 0
_________________________________________________________________
[<tf.Variable 'v2_conv2d/kernel:0' shape=(1, 1, 4, 64) dtype=float32, numpy=
array([[[[ 0.01920557, -0.26810384,  0.22631335,  0.09199455,
           0.26641685,  0.00082761,  0.16940999, -0.08342358,
           0.11300257,  0.04578683,  0.21995461, -0.07693768,
          -0.19567347, -0.09900907, -0.27615213,  0.11148515,
          -0.1818741 , -0.00957569, -0.18748233

In [ ]:
batch_size = 64

# set up paths
nih_cxr8_base = os.path.sep.join(['\\\\KRAMPUS-16', 'DataAll', 'NIH_Cxr8'])
nih_cxr8_csv = os.path.sep.join([nih_cxr8_base, 'Data_Entry_2017_v2020.csv'])
logging.info(f"Reading from {nih_cxr8_csv}")

# read the csv dataset
cxr_batched_ds = tf.data.experimental.make_csv_dataset(
    file_pattern=nih_cxr8_csv, 
    batch_size=batch_size, num_epochs=1, num_parallel_reads=20,
    shuffle=False)

# look at first 50
cxr_batched_ds = cxr_batched_ds.take(50)

# filter for No Finding
nofinding_cxrs = \
    cxr_batched_ds.filter(
        lambda item: tf.equal(item['Finding Labels'][0], 'No Finding'))

# show a few
for batch in nofinding_cxrs.take(3):
    logging.info("-----------")
    for item in batch:
        logging.info(f"{item}: {batch[item]}")

In [ ]:
# perform single epoch
for batch_x_in in train_dataset:
    with tf.GradientTape() as tape:
        batch_x_pred = autoencoder(batch_x_in)
        loss = vae_loss(z_mean, z_log_var, batch_x_in, batch_x_pred)
    grads = tape.gradient(loss, autoencoder.trainable_variables)
    optimizer.apply_gradients(zip(grads, autoencoder.trainable_variables))
    print(grads)